# Supplementary figure on alternate norms for $W$

Code to combine the results file into one DataFrame in `for_plots/` results folder:

```
>>> df = pd.read_hdf("df_w_stats_biopca.h5")
>>> df2 = pd.read_hdf("df_w_stats_ibcm.h5")
>>> df_both = pd.concat({"ibcm":df2, "biopca":df}, names=["Model"])
>>> df_both.to_hdf("df_w_pqnorms_stats_bothmodels.h5", key="df")
``` 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, json
import pandas as pd
import itertools
pj = os.path.join

In [ ]:
do_save_plots = True
# Resources
root_dir = pj("..", "..", "..")
data_folder = pj(root_dir, "results", "for_plots")
panels_folder = "panels/"
params_folder = pj(root_dir, "results", "common_params")

In [ ]:
# rcParams
with open(pj(params_folder, "olfaction_rcparams.json"), "r") as f:
    new_rcParams = json.load(f)
plt.rcParams.update(new_rcParams)

# color maps
with open(pj(params_folder, "back_colors.json"), "r") as f:
    all_back_colors = json.load(f)
back_color = all_back_colors["back_color"]
back_color_samples = all_back_colors["back_color_samples"]
back_palette = all_back_colors["back_palette"]

with open(pj(params_folder, "orn_colors.json"), "r") as f:
    orn_colors = json.load(f)
    
with open(pj(params_folder, "inhibitory_neuron_two_colors.json"), "r") as f:
    neuron_colors = np.asarray(json.load(f))
with open(pj(params_folder, "inhibitory_neuron_full_colors.json"), "r") as f:
    neuron_colors_full24 = np.asarray(json.load(f))
# Here, 32 neurons, need to make a new palette with same parameters
neuron_colors_full = np.asarray(sns.husl_palette(n_colors=32, h=0.01, s=0.9, l=0.4, as_cmap=False))

with open(pj(params_folder, "model_colors.json"), "r") as f:
    model_colors = json.load(f)
with open(pj(params_folder, "model_nice_names.json"), "r") as f:
    model_nice_names = json.load(f)

models = list(model_colors.keys())
print(models)

## Plotting function

In [ ]:
metrics_choice = [
    "s_norm_mean_reduction",
    "s_norm_variance_reduction",
    "s_norm_thirdmoment_reduction",
    "jaccard_mean",
    "jaccard_median",
    "jaccard_variance"
]
metrics_nice_names = dict(zip(metrics_choice, [
    r"Mean $\|\mathbf{y}\| /$ Mean $\|\mathbf{s}_{\mathrm{b}}\|$", 
    r"Var. $\|\mathbf{y}\| /$ Var. $\|\mathbf{s}_{\mathrm{b}}\|$", 
    r"$\mu_3$ $\|\mathbf{y}\| /$ $\mu_3$ $\|\mathbf{s}_{\mathrm{b}}\|$", 
    r"Mean  $J(z_\mathrm{mix}, z_\mathrm{new})$", 
    r"Median  $J(z_\mathrm{mix}, z_\mathrm{new})$", 
    r"Variance  $J(z_\mathrm{mix}, z_\mathrm{new})$"
]))

In [ ]:
def remove_exp0(s):
    return s[:-2] + s[-1]

In [ ]:
def main_bar_plot_w_norms(df_ibcm, df_pca):
    # Make a clearer plot now. Take the best (alpha, beta) combination
    # for each p, q and make barplots both models, and annotate
    # optimal alpha, beta for each bar.
    df_ibcm2 = df_ibcm.copy()
    df_pca2 = df_pca.copy()

    
    fig2, axes2 = plt.subplots(2, 3)
    fig2.set_size_inches(plt.rcParams["figure.figsize"][0]*3, plt.rcParams["figure.figsize"][1]*2)
    pq_combi = list(itertools.product(
        df_pca2["pnorm"].unique(), df_pca2["qnorm"].unique()
    ))
    qstr = "pnorm == @p and qnorm == @q"
    idx_lvls = ["alpha", "beta"]
    all_annotations = []
    bwd = 0.3  # bar width
    for i, m in enumerate(metrics_choice):
        increm = 0.025*df_pca2.loc[:, m].max()
        for x, (p, q) in enumerate(pq_combi):
            y_ibcm = df_ibcm2.query(qstr).set_index(idx_lvls)
            y_pca = df_pca2.query(qstr).set_index(idx_lvls)
            if m.startswith("s_norm"):
                yidx_ibcm = y_ibcm["s_norm_mean_reduction"].idxmin()
                yidx_pca = y_pca["s_norm_mean_reduction"].idxmin()
            else:
                yidx_ibcm = y_ibcm["jaccard_mean"].idxmax()
                yidx_pca = y_pca["jaccard_mean"].idxmax()
            lbl = model_nice_names["biopca"] if x == 0 else None
            axes2.flat[i].bar(x-bwd/2, y_pca.loc[yidx_pca, m], width=bwd,
                            color=model_colors.get("biopca"), label=lbl)
            ann = axes2.flat[i].annotate(
                ", ".join((remove_exp0("{:.0e}".format(a)) for a in yidx_pca)),
                xy=(x-bwd,y_pca.loc[yidx_pca, m]+increm), fontsize=5,
                rotation=90, ha="left"
            )
            all_annotations.append(ann)
            
            lbl = model_nice_names["ibcm"] if x == 0 else None
            axes2.flat[i].bar(x+bwd/2, y_ibcm.loc[yidx_ibcm, m], width=bwd, 
                            color=model_colors.get("ibcm"), label=lbl)
            ann = axes2.flat[i].annotate(
                ", ".join((remove_exp0("{:.0e}".format(a)) for a in yidx_ibcm)),
                xy=(x+bwd/4,y_ibcm.loc[yidx_ibcm, m]+increm), fontsize=5,
                rotation=90, ha="left"
            )
            all_annotations.append(ann)
        axes2.flat[i].set_xticks(range(len(pq_combi)))
        axes2.flat[i].set_xticklabels([(int(p), int(q)) for (p, q) in pq_combi])
        if (i - axes2.size) >= -3:
            axes2.flat[i].set_xlabel(r"$(p, q)$ combination")
        axes2.flat[i].set_ylabel(metrics_nice_names[m])
        for s in ["top", "right"]:
            axes2.flat[i].spines[s].set_visible(False)
    fig2.tight_layout()

    return [fig2, axes2, all_annotations]


# Load and plot simulations results statistics

In [ ]:
df_both = pd.read_hdf(pj(data_folder, "df_w_pqnorms_stats_bothmodels.h5"), key="df")

In [ ]:
figb, axb, annots = main_bar_plot_w_norms(df_both.xs("ibcm", level="Model"), df_both.xs("biopca", level="Model"))
axb.flat[1].legend(frameon=False, ncols=2, loc="lower right", bbox_to_anchor=(1, 1.1), columnspacing=1.0)
if do_save_plots:
    figb.savefig(pj(panels_folder, "supfig_wnorm_pq_bar_graphs.pdf"), transparent=True, 
                 bbox_inches="tight", bbox_extra_artists=tuple(annots))
plt.show()
plt.close()